In [9]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", message="The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.", category=FutureWarning)

In [10]:
DATA_PATH = 'D:\\0Code\\UNSW\\2024Kaggle\\03_NCA_Comp\\dataset\\'

### 1. 首先要打乱比赛结果(csv中分为WTeam和LTeam，没有明确标记结果)

In [11]:
RSresultW = DATA_PATH+'04_WRegularSeasonCompactResults.csv'
RSresultW_df = pd.read_csv(RSresultW)

In [12]:
RSresultW_df = RSresultW_df[RSresultW_df['Season']>=2010]
RSresultW_df = RSresultW_df.drop_duplicates(subset=['Season', 'WTeamID', 'LTeamID'])
RSresultW_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
55101,2010,11,3103,63,3237,49,H,0
55102,2010,11,3104,73,3399,68,N,0
55103,2010,11,3110,71,3224,59,A,0
55104,2010,11,3111,63,3267,58,A,0
55105,2010,11,3119,74,3447,70,H,1
...,...,...,...,...,...,...,...,...
130887,2024,114,3395,73,3403,52,H,0
130889,2024,114,3407,102,3111,69,H,0
130890,2024,114,3409,76,3396,67,A,0
130892,2024,114,3433,69,3348,59,A,0


In [13]:
NCAAresultW = DATA_PATH+'05_WNCAATourneyCompactResults.csv'
NCAAresultW_df = pd.read_csv(NCAAresultW)

In [14]:
NCAAresultW_df = NCAAresultW_df[NCAAresultW_df['Season']>=2010]
NCAAresultW_df = NCAAresultW_df.drop_duplicates(subset=['Season', 'WTeamID', 'LTeamID'])
NCAAresultW_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
756,2010,138,3124,69,3201,55,N,0
757,2010,138,3173,67,3395,66,N,0
758,2010,138,3181,72,3214,37,H,0
759,2010,138,3199,75,3256,61,H,0
760,2010,138,3207,62,3265,42,N,0
...,...,...,...,...,...,...,...,...
1578,2023,147,3376,86,3268,75,H,0
1579,2023,147,3439,84,3326,74,N,0
1580,2023,151,3234,77,3376,73,N,0
1581,2023,151,3261,79,3439,72,N,0


In [15]:
# 定义函数以随机交换队伍和分数
def shuffle_teams(row):
    if np.random.rand() > 0.5:
        # 如果随机数大于0.5，交换胜利和失败队伍的ID及其分数
        return pd.Series({'Year':row['Season'], 'Team1': row['LTeamID'], 'Score1': row['LScore'], 
                          'Team2': row['WTeamID'], 'Score2': row['WScore'], 
                          'Result': 1 if row['LScore'] > row['WScore'] else (2 if row['LScore'] < row['WScore'] else 0)})
    else:
        return pd.Series({'Year':row['Season'], 'Team1': row['WTeamID'], 'Score1': row['WScore'], 
                          'Team2': row['LTeamID'], 'Score2': row['LScore'], 
                          'Result': 1 if row['WScore'] > row['LScore'] else (2 if row['WScore'] < row['LScore'] else 0)})


In [19]:
RSresultW_df2 = RSresultW_df.apply(shuffle_teams, axis=1)
RSresultW_df2 = RSresultW_df2.drop(['Score1', 'Score2'], axis=1)
RSresultW_df2['TeamPair'] = RSresultW_df2.apply(lambda row: tuple(sorted([row['Team1'], row['Team2']])), axis=1)
RSresultW_df3 = RSresultW_df2.drop_duplicates(subset=['Year','TeamPair'])
# RSresultW_df3 = RSresultW_df3.drop(columns='TeamPair')
RSresultW_df3

,Year,Team1,Team2,Result,TeamPair
55101,2010,3237,3103,2,"(3103, 3237)"
55102,2010,3104,3399,1,"(3104, 3399)"
55103,2010,3224,3110,2,"(3110, 3224)"
55104,2010,3267,3111,2,"(3111, 3267)"
55105,2010,3119,3447,1,"(3119, 3447)"
...,...,...,...,...,...
130884,2024,3455,3374,2,"(3374, 3455)"
130886,2024,3247,3387,2,"(3247, 3387)"
130889,2024,3111,3407,2,"(3111, 3407)"
130892,2024,3433,3348,1,"(3348, 3433)"


In [20]:
NCAAresultW_df2 = NCAAresultW_df.apply(shuffle_teams, axis=1)
NCAAresultW_df2 = NCAAresultW_df2.drop(['Score1', 'Score2'], axis=1)
NCAAresultW_df2['TeamPair'] = NCAAresultW_df2.apply(lambda row: tuple(sorted([row['Team1'], row['Team2']])), axis=1)
NCAAresultW_df2 = NCAAresultW_df2.drop_duplicates(subset=['Year','TeamPair'])
NCAAresultW_df2

,Year,Team1,Team2,Result,TeamPair
756,2010,3201,3124,2,"(3124, 3201)"
757,2010,3173,3395,1,"(3173, 3395)"
758,2010,3181,3214,1,"(3181, 3214)"
759,2010,3256,3199,2,"(3199, 3256)"
760,2010,3207,3265,1,"(3207, 3265)"
...,...,...,...,...,...
1578,2023,3376,3268,1,"(3268, 3376)"
1579,2023,3439,3326,1,"(3326, 3439)"
1580,2023,3234,3376,1,"(3234, 3376)"
1581,2023,3439,3261,2,"(3261, 3439)"


In [23]:
Wresult_df = pd.concat([RSresultW_df3, NCAAresultW_df2], ignore_index=True)
Wresult_df = Wresult_df.drop_duplicates(subset=['Year','TeamPair'],keep='last')
Wresult_df = Wresult_df.drop(columns='TeamPair')
Wresult_df

,Year,Team1,Team2,Result
0,2010,3237,3103,2
1,2010,3104,3399,1
2,2010,3224,3110,2
3,2010,3267,3111,2
4,2010,3119,3447,1
...,...,...,...,...
54990,2023,3376,3268,1
54991,2023,3439,3326,1
54992,2023,3234,3376,1
54993,2023,3439,3261,2


### 2. 根据Year和TeamID将ID转变为详细数据信息

In [24]:
Wdata = pd.read_csv(DATA_PATH+'001_Wdata.csv')
Wdata.head(5)

,TeamID,Season,Score,WinPct,FGM,FGA,FGM3,FGA3,FTM,FTA,...,ScoreVar,OppScore,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,OppStl,OppBlk
0,3103,2010,62.233333,0.566667,22.233333,55.266667,3.933333,12.466667,13.833333,19.600000,...,87.219540,59.633333,20.966667,54.966667,4.833333,15.166667,12.866667,18.833333,9.233333,3.800000
1,3104,2010,64.137931,0.379310,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,...,171.408867,67.000000,23.896552,60.137931,5.103448,16.517241,14.103448,21.965517,8.758621,3.586207
2,3110,2010,60.428571,0.619048,21.571429,53.904762,4.666667,15.285714,12.619048,16.809524,...,132.657143,54.380952,20.000000,52.904762,4.095238,14.190476,10.285714,15.000000,5.714286,2.666667
3,3111,2010,66.419355,0.612903,25.774194,59.483871,4.290323,12.387097,10.580645,16.612903,...,92.651613,63.838710,21.516129,59.193548,6.129032,18.645161,14.677419,20.548387,9.741935,3.516129
4,3119,2010,52.291667,0.333333,19.166667,53.541667,4.625000,16.916667,9.333333,13.875000,...,90.737319,58.750000,20.750000,54.750000,4.541667,13.166667,12.708333,18.458333,7.875000,3.166667


In [25]:
Wdata[['TeamID','Season','Score']]

,TeamID,Season,Score
0,3103,2010,62.233333
1,3104,2010,64.137931
2,3110,2010,60.428571
3,3111,2010,66.419355
4,3119,2010,52.291667
...,...,...,...
5235,3141,2024,60.583333
5236,3476,2024,51.846154
5237,3214,2024,55.041667
5238,3152,2024,57.192308


In [26]:
# df1 = RSresultW_df2[RSresultW_df2['Year']==2010]
# df2 = Wdata[Wdata['Season']==2010]

df1 = RSresultW_df2.copy()
df2 = Wdata.copy()

In [27]:
merged_df1 = pd.merge(df1, df2, left_on=['Year', 'Team1'], right_on=['Season', 'TeamID'], how='left')
merged_df1 = merged_df1.rename(columns={
    'Score': 'Score1', 
    'FGM': 'FGM1', 
    'FGA': 'FGA1', 
    'FGM3': 'FGM31', 
    'FGA3': 'FGA31', 
    'FTM': 'FTM1', 
    'FTA': 'FTA1', 
    'OR': 'OR1', 
    'DR': 'DR1', 
    'Ast': 'Ast1', 
    'TO': 'TO1', 
    'Stl': 'Stl1', 
    'Blk': 'Blk1', 
    'ScoreVar': 'ScoreVar1', 
    'OppScore': 'OppScore1', 
    'OppFGM': 'OppFGM1', 
    'OppFGA': 'OppFGA1', 
    'OppFGM3': 'OppFGM31', 
    'OppFGA3': 'OppFGA31', 
    'OppFTM': 'OppFTM1', 
    'OppFTA': 'OppFTA1', 
    'OppStl': 'OppStl1', 
    'OppBlk': 'OppBlk1', 
    'WinPct': 'WinPct1'
})


In [28]:
merged_df1

,Year,Team1,Team2,Result,TeamPair,TeamID,Season,Score1,WinPct1,FGM1,...,ScoreVar1,OppScore1,OppFGM1,OppFGA1,OppFGM31,OppFGA31,OppFTM1,OppFTA1,OppStl1,OppBlk1
0,2010,3237,3103,2,"(3103, 3237)",3237.0,2010.0,51.321429,0.071429,18.571429,...,103.707672,69.964286,24.428571,57.071429,5.964286,16.071429,15.142857,21.392857,11.107143,2.642857
1,2010,3104,3399,1,"(3104, 3399)",3104.0,2010.0,64.137931,0.379310,24.724138,...,171.408867,67.000000,23.896552,60.137931,5.103448,16.517241,14.103448,21.965517,8.758621,3.586207
2,2010,3224,3110,2,"(3110, 3224)",3224.0,2010.0,56.862069,0.517241,20.137931,...,92.123153,59.517241,21.068966,53.965517,5.275862,17.413793,12.103448,17.482759,7.137931,2.965517
3,2010,3267,3111,2,"(3111, 3267)",3267.0,2010.0,59.310345,0.448276,23.206897,...,115.793103,62.068966,22.379310,56.310345,5.965517,15.586207,11.344828,17.448276,8.379310,2.413793
4,2010,3119,3447,1,"(3119, 3447)",3119.0,2010.0,52.291667,0.333333,19.166667,...,90.737319,58.750000,20.750000,54.750000,4.541667,13.166667,12.708333,18.458333,7.875000,3.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60926,2024,3395,3403,1,"(3395, 3403)",3395.0,2024.0,70.740741,0.666667,25.074074,...,179.968661,60.888889,22.296296,61.185185,5.555556,20.370370,10.740741,14.888889,6.888889,2.000000
60927,2024,3111,3407,2,"(3111, 3407)",3111.0,2024.0,72.379310,0.517241,24.862069,...,153.743842,72.724138,26.482759,64.000000,5.620690,17.448276,14.137931,19.655172,8.103448,3.517241
60928,2024,3396,3409,2,"(3396, 3409)",3396.0,2024.0,69.642857,0.607143,26.428571,...,144.904762,62.928571,21.785714,55.607143,4.500000,15.964286,14.857143,20.035714,5.750000,3.535714
60929,2024,3433,3348,1,"(3348, 3433)",3433.0,2024.0,63.285714,0.857143,22.571429,...,59.322751,52.678571,19.500000,53.964286,4.392857,18.678571,9.285714,13.035714,6.357143,2.642857


In [29]:
merged_df2 = pd.merge(df1, df2, left_on=['Year', 'Team2'], right_on=['Season', 'TeamID'], how='left')
merged_df2 = merged_df2.rename(columns={
    'Score': 'Score2', 
    'FGM': 'FGM2', 
    'FGA': 'FGA2', 
    'FGM3': 'FGM32', 
    'FGA3': 'FGA32', 
    'FTM': 'FTM2', 
    'FTA': 'FTA2', 
    'OR': 'OR2', 
    'DR': 'DR2', 
    'Ast': 'Ast2', 
    'TO': 'TO2', 
    'Stl': 'Stl2', 
    'Blk': 'Blk2', 
    'ScoreVar': 'ScoreVar2', 
    'OppScore': 'OppScore2', 
    'OppFGM': 'OppFGM2', 
    'OppFGA': 'OppFGA2', 
    'OppFGM3': 'OppFGM32', 
    'OppFGA3': 'OppFGA32', 
    'OppFTM': 'OppFTM2', 
    'OppFTA': 'OppFTA2', 
    'OppStl': 'OppStl2', 
    'OppBlk': 'OppBlk2', 
    'WinPct': 'WinPct2'
})

In [30]:
merged_df2

,Year,Team1,Team2,Result,TeamPair,TeamID,Season,Score2,WinPct2,FGM2,...,ScoreVar2,OppScore2,OppFGM2,OppFGA2,OppFGM32,OppFGA32,OppFTM2,OppFTA2,OppStl2,OppBlk2
0,2010,3237,3103,2,"(3103, 3237)",3103.0,2010.0,62.233333,0.566667,22.233333,...,87.219540,59.633333,20.966667,54.966667,4.833333,15.166667,12.866667,18.833333,9.233333,3.800000
1,2010,3104,3399,1,"(3104, 3399)",3399.0,2010.0,65.241379,0.448276,22.965517,...,120.332512,67.655172,23.034483,56.482759,4.517241,13.758621,17.068966,25.206897,6.344828,3.275862
2,2010,3224,3110,2,"(3110, 3224)",3110.0,2010.0,60.428571,0.619048,21.571429,...,132.657143,54.380952,20.000000,52.904762,4.095238,14.190476,10.285714,15.000000,5.714286,2.666667
3,2010,3267,3111,2,"(3111, 3267)",3111.0,2010.0,66.419355,0.612903,25.774194,...,92.651613,63.838710,21.516129,59.193548,6.129032,18.645161,14.677419,20.548387,9.741935,3.516129
4,2010,3119,3447,1,"(3119, 3447)",3447.0,2010.0,60.333333,0.300000,22.200000,...,74.160920,67.066667,22.666667,55.300000,6.133333,16.866667,15.600000,22.066667,9.666667,3.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60926,2024,3395,3403,1,"(3395, 3403)",3403.0,2024.0,65.266667,0.533333,22.533333,...,203.581609,62.333333,22.966667,54.700000,5.433333,18.200000,10.966667,15.733333,7.900000,3.666667
60927,2024,3111,3407,2,"(3111, 3407)",3407.0,2024.0,79.892857,0.642857,30.571429,...,137.876984,75.964286,27.750000,68.464286,7.321429,23.107143,13.142857,18.428571,7.464286,2.785714
60928,2024,3396,3409,2,"(3396, 3409)",3409.0,2024.0,70.448276,0.724138,24.793103,...,177.827586,67.103448,24.379310,65.413793,5.620690,19.517241,12.724138,17.310345,7.413793,2.068966
60929,2024,3433,3348,1,"(3348, 3433)",3348.0,2024.0,64.066667,0.566667,24.133333,...,151.512644,57.033333,21.133333,53.133333,5.500000,16.300000,9.266667,12.566667,6.500000,2.600000


In [31]:
# 合并merged_df1和merged_df2
final_df = pd.merge(merged_df1, merged_df2, on=['Year', 'Team1', 'Team2', 'Result'], how='left')

# 选择所需的列
columns_to_keep = [
    'Score1', 'FGM1', 'FGA1', 'FGM31', 'FGA31', 'FTM1', 'FTA1', 'OR1', 'DR1', 
    'Ast1', 'TO1', 'Stl1', 'Blk1', 'ScoreVar1', 'OppScore1', 'OppFGM1', 'OppFGA1', 
    'OppFGM31', 'OppFGA31', 'OppFTM1', 'OppFTA1', 'OppStl1', 'OppBlk1', 'WinPct1',
    'Score2', 'FGM2', 'FGA2', 'FGM32', 'FGA32', 'FTM2', 'FTA2', 'OR2', 'DR2', 
    'Ast2', 'TO2', 'Stl2', 'Blk2', 'ScoreVar2', 'OppScore2', 'OppFGM2', 'OppFGA2', 
    'OppFGM32', 'OppFGA32', 'OppFTM2', 'OppFTA2', 'OppStl2', 'OppBlk2', 'WinPct2', 'Result'
]

final_df = final_df[columns_to_keep]

In [32]:
final_df

,Score1,FGM1,FGA1,FGM31,FGA31,FTM1,FTA1,OR1,DR1,Ast1,...,OppFGM2,OppFGA2,OppFGM32,OppFGA32,OppFTM2,OppFTA2,OppStl2,OppBlk2,WinPct2,Result
0,51.321429,18.571429,50.607143,2.714286,11.964286,11.464286,17.785714,7.500000,24.500000,10.500000,...,20.966667,54.966667,4.833333,15.166667,12.866667,18.833333,9.233333,3.800000,0.566667,2
1,64.137931,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,14.363636,28.000000,17.181818,...,23.034483,56.482759,4.517241,13.758621,17.068966,25.206897,6.344828,3.275862,0.448276,1
2,56.862069,20.137931,58.000000,2.758621,11.413793,13.827586,19.862069,14.733333,25.200000,11.200000,...,20.000000,52.904762,4.095238,14.190476,10.285714,15.000000,5.714286,2.666667,0.619048,2
3,59.310345,23.206897,57.482759,2.655172,9.275862,10.241379,18.172414,14.923077,27.769231,17.153846,...,21.516129,59.193548,6.129032,18.645161,14.677419,20.548387,9.741935,3.516129,0.612903,2
4,52.291667,19.166667,53.541667,4.625000,16.916667,9.333333,13.875000,13.500000,27.125000,12.125000,...,22.666667,55.300000,6.133333,16.866667,15.600000,22.066667,9.666667,3.666667,0.300000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60926,70.740741,25.074074,57.296296,9.518519,26.000000,11.074074,15.185185,9.555556,27.888889,18.777778,...,22.966667,54.700000,5.433333,18.200000,10.966667,15.733333,7.900000,3.666667,0.533333,1
60927,72.379310,24.862069,64.137931,7.517241,23.793103,15.137931,22.137931,12.733333,23.533333,14.333333,...,27.750000,68.464286,7.321429,23.107143,13.142857,18.428571,7.464286,2.785714,0.642857,2
60928,69.642857,26.428571,67.357143,5.464286,18.571429,11.321429,16.000000,13.470588,23.705882,14.588235,...,24.379310,65.413793,5.620690,19.517241,12.724138,17.310345,7.413793,2.068966,0.724138,2
60929,63.285714,22.571429,54.714286,5.500000,15.250000,12.642857,17.571429,10.541667,23.541667,10.583333,...,21.133333,53.133333,5.500000,16.300000,9.266667,12.566667,6.500000,2.600000,0.566667,1


In [33]:
final_df2 = final_df.dropna()

In [ ]:
final_df2.to_csv('002_Wdataset.csv', index=False)